In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("datasets/train.csv",encoding='latin')
print(train.shape)
train.head()

FileNotFoundError: File b'datasets/train.csv' does not exist

In [2]:
#lets collect more tweets
positives = open("datasets/positive.txt","r",encoding='latin').read().split('\n')
negatives = open("datasets/negative.txt","r",encoding='latin').read().split('\n')
print(len(positives)+len(negatives))
positives[:5]

10662


['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . ',
 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth . ',
 'effective but too-tepid biopic',
 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start . ',
 "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one . "]

In [3]:
train.drop("ItemID",axis=1,inplace=True)
train  = train.append(pd.DataFrame({
            'Sentiment':[1]*len(positives),
            'SentimentText':positives
            }),ignore_index=True)
train = train.append(pd.DataFrame({
            'Sentiment':[0]*len(negatives),
            'SentimentText':negatives
            }),ignore_index=True)
train.shape

(110651, 2)

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))
newtweets = []
tokens_list = []
def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in nltk.pos_tag(nltk.word_tokenize(sentence)):
        if word in stop_words:
            continue
        if tag.startswith('V'):
            yield wnl.lemmatize(word.lower(), pos='v')
        elif tag.startswith('J') or tag.startswith('R'):
            yield wnl.lemmatize(word.lower(), pos='a')
        #elif tag.startswith('NN') or tag.startswith('NNP'):
        #   yield wnl.lemmatize(word.lower(), pos='n')
            
for sentence in train.SentimentText.values:
    temp = list(lemmatize_all(sentence))
    #tokens_list.append(temp)
    newtweets.append(' '.join(temp))

NameError: name 'train' is not defined

In [5]:
newtweets[:10]

['sad',
 'miss',
 'omg already',
 ".. sooo im 've suposed get put",
 'think cheat',
 'worry much',
 '',
 '-|',
 'hand uniform miss already',
 'wonder']

In [6]:
i=0
chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '
for tweet in newtweets:
    newtweets[i] = ''.join(c for c in tweet if c in chars).strip()
    i+=1
newtweets[:10]

['sad',
 'miss',
 'omg already',
 'sooo im ve suposed get put',
 'think cheat',
 'worry much',
 '',
 '',
 'hand uniform miss already',
 'wonder']

In [7]:
train['SentimentText'] = np.array(newtweets)

In [8]:
train.drop(train[train.SentimentText.str.len()<2].index,axis=0,inplace=True)
train.head(10)

,Sentiment,SentimentText
0,0,sad
1,0,miss
2,1,omg already
3,0,sooo im ve suposed get put
4,0,think cheat
5,0,worry much
8,1,hand uniform miss already
9,1,wonder
10,0,think
12,0,suck far


In [9]:
train = train.sample(frac=1)
train.groupby('Sentiment').size()

Sentiment
0    45958
1    55806
dtype: int64

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit(train.SentimentText)
X = count_vect.transform(train.SentimentText)
X.shape

(101764, 32814)

In [11]:
import pickle
countvect = open("count_vect.pickle","wb")
pickle.dump(count_vect,countvect)
countvect.close()

In [12]:
Y = train['Sentiment']

In [13]:
# TF-IDF i.e Term Frequency times inverse document frequency.
# TF:  It will give more weightage to longer documents than shorter documents.
# IDF: Finally, we can even reduce the weightage of more common words like (the, is, an etc.) 
#      which occurs in all document.
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(X)
X = tfidf_transformer.transform(X)
X.shape

(101764, 32814)

In [14]:
import pickle
tfidftransformer = open("tfidf_transformer.pickle","wb")
pickle.dump(tfidf_transformer,tfidftransformer)
tfidftransformer.close()

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_validation,Y_train,Y_validation = train_test_split(X,Y,test_size=0.2)

In [16]:
from sklearn.naive_bayes import MultinomialNB
multinomial_nb = MultinomialNB().fit(X_train, Y_train )
print("Accuracy : {}".format(multinomial_nb.score(X_validation,Y_validation)))
mnb = open("multinomialnb.pickle","wb")
pickle.dump(multinomial_nb,mnb)
mnb.close()

Accuracy : 0.6964083918832604


In [17]:
from sklearn.naive_bayes import BernoulliNB
bernoulli_nb = BernoulliNB().fit(X_train, Y_train)
print("Accuracy : {}".format(bernoulli_nb.score(X_validation,Y_validation)))
bnb = open("bernoulli_nb.pickle","wb")
pickle.dump(bernoulli_nb,bnb)
bnb.close()

Accuracy : 0.7020095317643591


In [21]:
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression().fit(X_train, Y_train)
print("Accuracy : {}".format(logistic_regression.score(X_validation,Y_validation)))
lr = open("logistic_regression.pickle","wb")
pickle.dump(logistic_regression,lr)
lr.close()

Accuracy : 0.705841890630374


In [18]:
#from sklearn.svm import SVC
#SVC = SVC(njobs=-1).fit(X_train,Y_train)
#print("Accuracy : {}".format(SVC.score(X_validation,Y_validation)))
#svc = open("svc.pickle","wb")
#pickle.dump(SVC,svc)
#svc.close()

In [12]:
import pickle
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
multinomial_nb = pickle.load(open("multinomialnb.pickle","rb"))
bernoulli_nb = pickle.load(open("bernoulli_nb.pickle","rb"))
logistic_regression = pickle.load(open("logistic_regression.pickle","rb"))
count_vect = pickle.load(open("count_vect.pickle","rb"))
tfidf_transformer = pickle.load(open("tfidf_transformer.pickle","rb"))
from statistics import mode
class VoteClassifier():
    def __init__(self,*classifiers):
        self.classifiers = classifiers
        
    def predict(self,tweet):
        X = count_vect.transform(np.array([tweet]))
        X = tfidf_transformer.transform(X)
        votes = []
        for c in self.classifiers:
            v = c.predict(X)
            votes.append(v[0])
        return mode(votes)
    
    def confidence(self,tweet):
        X = count_vect.transform(np.array([tweet]))
        X = tfidf_transformer.transform(X)
        votes = []
        for c in self.classifiers:
            v = c.predict(X)
            votes.append(v[0])
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [13]:
votingClassifier = VoteClassifier(multinomial_nb,bernoulli_nb,logistic_regression)

In [14]:
print(votingClassifier.predict("sad"),votingClassifier.confidence("sad"))

0 1.0


In [15]:
test = pd.read_csv('../datasets/test.csv',encoding='latin')
test.head()

,ItemID,SentimentText
0,1,is so sad for my APL frie...
1,2,I missed the New Moon trail...
2,3,omg its already 7:30 :O
3,4,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,i think mi bf is cheating on me!!! ...


In [22]:
ids = test['ItemID']
X = test['SentimentText']
X.shape

(299989,)

In [20]:
def value(tweet):
    chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '
    tweet = ' '.join(lemmatize_all(tweet))
    tweet = ''.join(c for c in tweet if c in chars).strip()
    return votingClassifier.predict(tweet)

vectorizer = np.vectorize(value)

In [21]:
predictions = vectorizer(X)
len(predictions)

299989

In [23]:
predict = pd.DataFrame({
    "ItemId":ids.values,
    "Sentiment":predictions
})
predict.to_csv("sol.csv",index=False)